### Doctor's duty roster###
This purpose of this app is to automatically assign OFM shifts to doctors presented at hospital during that month.
(OFM stands for occupational-family-medicine). The OFM shifts are between 8:30-16:30, every weekday except for Tuesday.

In [1]:
#1. Get the dates of every weekdays of that month accordingly - assign it to a dictionary *weekday_date*
#2. Create a list with all the dates of that month - *all_dates*

In [1]:
from datetime import date, timedelta
import pandas as pd
import numpy as np

weekday_dict = {0:'monday', 1:'tuesday', 2:'wednesday', 3:'thursday', 4:'friday', 5:'saturday', 6:'sunday'}
weekday_date = {weekday:[] for weekday in weekday_dict.values()}

def get_dates (year, month):
    d = date(year, month, 1) #1 = first day of the month, do not change
    while d.month == month:
        day_of_week = weekday_dict[d.weekday()]
        weekday_date[day_of_week].append(d)
        d += timedelta(days = 1)
    return weekday_date

"""Change month here"""
weekday_date = get_dates(2020,11)


all_dates = []
for week_dates in weekday_date.values():
    all_dates = all_dates + [i for i in week_dates]
all_dates.sort()

In [2]:
weekday_date

{'monday': [datetime.date(2020, 11, 2),
  datetime.date(2020, 11, 9),
  datetime.date(2020, 11, 16),
  datetime.date(2020, 11, 23),
  datetime.date(2020, 11, 30)],
 'tuesday': [datetime.date(2020, 11, 3),
  datetime.date(2020, 11, 10),
  datetime.date(2020, 11, 17),
  datetime.date(2020, 11, 24)],
 'wednesday': [datetime.date(2020, 11, 4),
  datetime.date(2020, 11, 11),
  datetime.date(2020, 11, 18),
  datetime.date(2020, 11, 25)],
 'thursday': [datetime.date(2020, 11, 5),
  datetime.date(2020, 11, 12),
  datetime.date(2020, 11, 19),
  datetime.date(2020, 11, 26)],
 'friday': [datetime.date(2020, 11, 6),
  datetime.date(2020, 11, 13),
  datetime.date(2020, 11, 20),
  datetime.date(2020, 11, 27)],
 'saturday': [datetime.date(2020, 11, 7),
  datetime.date(2020, 11, 14),
  datetime.date(2020, 11, 21),
  datetime.date(2020, 11, 28)],
 'sunday': [datetime.date(2020, 11, 1),
  datetime.date(2020, 11, 8),
  datetime.date(2020, 11, 15),
  datetime.date(2020, 11, 22),
  datetime.date(2020, 11, 

In [3]:
all_dates

[datetime.date(2020, 11, 1),
 datetime.date(2020, 11, 2),
 datetime.date(2020, 11, 3),
 datetime.date(2020, 11, 4),
 datetime.date(2020, 11, 5),
 datetime.date(2020, 11, 6),
 datetime.date(2020, 11, 7),
 datetime.date(2020, 11, 8),
 datetime.date(2020, 11, 9),
 datetime.date(2020, 11, 10),
 datetime.date(2020, 11, 11),
 datetime.date(2020, 11, 12),
 datetime.date(2020, 11, 13),
 datetime.date(2020, 11, 14),
 datetime.date(2020, 11, 15),
 datetime.date(2020, 11, 16),
 datetime.date(2020, 11, 17),
 datetime.date(2020, 11, 18),
 datetime.date(2020, 11, 19),
 datetime.date(2020, 11, 20),
 datetime.date(2020, 11, 21),
 datetime.date(2020, 11, 22),
 datetime.date(2020, 11, 23),
 datetime.date(2020, 11, 24),
 datetime.date(2020, 11, 25),
 datetime.date(2020, 11, 26),
 datetime.date(2020, 11, 27),
 datetime.date(2020, 11, 28),
 datetime.date(2020, 11, 29),
 datetime.date(2020, 11, 30)]

### create date object for the holidays we wish to exclude

[ลิงค์วันหยุด](https://hilight.kapook.com/view/188249)

In [4]:
holidays = []
def create_holidays(year, month, day):
    d = date(year, month, day)
    holidays.append(d)
    return holidays

"""Change month here"""
# create_holidays(2020,7,6)
create_holidays(2020,11, 19)
create_holidays(2020,11,20)


print(holidays)

[datetime.date(2020, 11, 19), datetime.date(2020, 11, 20)]


In [5]:
# remove the holidays and unwanted weekdays

In [6]:
def remove_dates(all_dates, unwanted_weekdays, holidays): #unwanted = list-of-list of unwanted weekdays
#     OFM = all_dates- sundays - saturdays - tuesdays 
    OFM = all_dates
    for weekday in unwanted_weekdays:
        OFM = [date for date in OFM if date not in weekday_date[weekday]]
    OFM = [date for date in OFM if date not in holidays]
    return OFM

unwanted_weekdays = ['sunday','saturday','tuesday']
OFM_dates = remove_dates(all_dates, unwanted_weekdays, holidays)


print('Total number of days needed to fill =', len(OFM_dates), '\n')
print('The OFM dates are:')
OFM_dates

Total number of days needed to fill = 15 

The OFM dates are:


[datetime.date(2020, 11, 2),
 datetime.date(2020, 11, 4),
 datetime.date(2020, 11, 5),
 datetime.date(2020, 11, 6),
 datetime.date(2020, 11, 9),
 datetime.date(2020, 11, 11),
 datetime.date(2020, 11, 12),
 datetime.date(2020, 11, 13),
 datetime.date(2020, 11, 16),
 datetime.date(2020, 11, 18),
 datetime.date(2020, 11, 23),
 datetime.date(2020, 11, 25),
 datetime.date(2020, 11, 26),
 datetime.date(2020, 11, 27),
 datetime.date(2020, 11, 30)]

### Get the list of intern doctors' names
The names are stored in a csv file(exported from Excel)


In [7]:
opened_file = open('Book1.csv')
read_file = pd.read_csv("Book1.csv", encoding = 'utf-8' )
data = read_file.iloc[:,0:2]

data

,Name,Fix
0,Int3 ภาณุวัฒน์,Med
1,Int1 ชลกร,Med
2,Int1 วรธน,Med
3,Int1 ก้องภพ,Med
4,Int1 ยุตล์,Med
5,Int1 เภาลีนา,Med
6,Int1 วัชราภรณ์,Med
7,Int1 ภัทร์นฤน,Med
8,Int1 ณัฐฌา,Med
9,Int1 ชิชญา,Med


The names looked like this : 
>Int3 ภาณุวัฒน์, Med

So we need to separate the year indicator (Int3 indicates intern year 3) from the name. 

In [29]:
year_list = []
name_list = []
index_list = list(range(len(data)))
# splitting int from name
for i in read_file.iloc[:,0]:
    year,name = i.split()
    name = name.replace(' ','')
    year_list.append(year)
    name_list.append(name)
    
    
# create a pd with intern year as column and insert it back as first column in the main file (df)
df = {'Index': index_list,
      'Year': year_list,
      'Name':name_list} 
df = pd.DataFrame(df) 
df.insert(2, 'Fix', data['Fix'], True) 

# แก้ Bug คนที่ลาแล้วไม่มี year นำหน้า
L_df = df.copy(deep=True)
df


,Index,Year,Fix,Name
0,0,Int3,Med,ภาณุวัฒน์
1,1,Int1,Med,ชลกร
2,2,Int1,Med,วรธน
3,3,Int1,Med,ก้องภพ
4,4,Int1,Med,ยุตล์
5,5,Int1,Med,เภาลีนา
6,6,Int1,Med,วัชราภรณ์
7,7,Int1,Med,ภัทร์นฤน
8,8,Int1,Med,ณัฐฌา
9,9,Int1,Med,ชิชญา


In [9]:
# Create dictionary for later use
index_to_name = {i:n for i,n in enumerate(df['Name'])}
index_to_name

{0: 'ภาณุวัฒน์',
 1: 'ชลกร',
 2: 'วรธน',
 3: 'ก้องภพ',
 4: 'ยุตล์',
 5: 'เภาลีนา',
 6: 'วัชราภรณ์',
 7: 'ภัทร์นฤน',
 8: 'ณัฐฌา',
 9: 'ชิชญา',
 10: 'ณัฐชัย',
 11: 'พิชญา',
 12: 'วชิรภัทร์',
 13: 'เฉลิมเกียรติ',
 14: 'วรัญญา',
 15: 'ภาสกร',
 16: 'ปารินทร์',
 17: 'กมลนพ',
 18: 'นพรัตน์',
 19: 'เอนกโชค',
 20: 'ณัชชา',
 21: 'ชญานิศ',
 22: 'นภสร',
 23: 'ณิชารีย์',
 24: 'ปภาวดี',
 25: 'กฤตพร',
 26: 'ณภัทร',
 27: 'แพรววนิต',
 28: 'วัณย์วรยา',
 29: 'ชญากาณฑ์',
 30: 'ณิชญาดา',
 31: 'นัทธมน',
 32: 'ณัฐภัสสร',
 33: 'พิชชาภา',
 34: 'วีรภัทร',
 35: 'สรศักย์',
 36: 'ศุภาพิชญ์',
 37: 'นวลพรรณ',
 38: 'ศศิชา',
 39: 'จิดาภา',
 40: 'วิจิตรกานต์พัฒนา',
 41: 'พิชชาพร',
 42: 'สิรินญา',
 43: 'บุญฤทธิ์',
 44: 'พุฒิพงศ์',
 45: 'อภิชญา',
 46: 'เจษฎากร',
 47: 'ปุณยนุช',
 48: 'นิสากร',
 49: 'ศิรวิชญ์',
 50: 'นภดล',
 51: 'ปรียาภรณ์'}

### Remove names

* Intern-1s aren't included in OFM shifts, so we need to remove them
* Remove resigned
* Remove those who are not present at the hospital for this month

In [10]:
# remove intern 1
df.drop(df[df.loc[:,'Year'] == 'Int1'].index, inplace = True)

# remove resigned and not present
resigned = ['ศุภาพิชญ์'] 
not_present = ["ภาณุวัฒน์","สรศักย์",'ปารินทร์']
# สรศักย์อยู่เยอะแล้วเลยใส่ไว้
other_OPD = []

for name in (not_present + resigned + other_OPD):
    df.drop(df[df.loc[:,'Name'] == name].index, inplace = True)


print("Resigned :{}".format(resigned), "Not_preset :{}".format(not_present), "Other OPD: {}".format(other_OPD))
print("There are {0} interns present, มีอินเทิร์นเหลือทั้งหมด {0} คน".format(len(df)))
df

Resigned :['ศุภาพิชญ์'] Not_preset :['ภาณุวัฒน์', 'สรศักย์', 'ปารินทร์'] Other OPD: []
There are 17 interns present, มีอินเทิร์นเหลือทั้งหมด 17 คน


,Index,Year,Fix,Name
10,10,Int2,Sx,ณัฐชัย
15,15,Int3,Ortho,ภาสกร
19,19,Int3,Ped,เอนกโชค
20,20,Int3,Ped,ณัชชา
21,21,Int2,Ped,ชญานิศ
22,22,Int2,Ped,นภสร
23,23,Int2,Ped,ณิชารีย์
24,24,Int2,Ped,ปภาวดี
31,31,Int3,Eye,นัทธมน
32,32,Int3,Eye,ณัฐภัสสร


In [11]:
import pickle
def try_open_saves(month):
    month_file = str(month).lower()+'_count'
    print('searching for',month_file,'..retrieving data..','\n')
        
    try:
        with open(month_file, 'rb') as f:
            old_dict = pickle.load(f)   
            for key in old_dict:
                print('The number of shifts so far...{}..x..{}..times'.format(key, old_dict[key]))
    except:
        old_dict = '{}'
        print('No roster was made before, nothing to display - ยังไม่เคยมีการจัดเวรมาก่อน')
    return old_dict

print('loading saves..')
old_dict = try_open_saves("October")

loading saves..
searching for october_count ..retrieving data.. 

The number of shifts so far...นภสร..x..5..times
The number of shifts so far...พิชชาภา..x..5..times
The number of shifts so far...ชญานิศ..x..5..times
The number of shifts so far...สรศักย์..x..6..times
The number of shifts so far...อภิชญา..x..5..times
The number of shifts so far...ณัฐชัย..x..5..times
The number of shifts so far...ณิชารีย์..x..5..times
The number of shifts so far...ภาสกร..x..5..times
The number of shifts so far...บุญฤทธิ์..x..3..times
The number of shifts so far...ภาณุวัฒน์..x..5..times
The number of shifts so far...ณัชชา..x..5..times
The number of shifts so far...นัทธมน..x..4..times
The number of shifts so far...ณัฐภัสสร..x..5..times
The number of shifts so far...เอนกโชค..x..5..times
The number of shifts so far...ปภาวดี..x..5..times
The number of shifts so far...ปารินทร์..x..5..times
The number of shifts so far...พุฒิพงศ์..x..5..times


In [27]:
# subs = Med, Rehab, X-ray, ENT, Eye, Ped, Ortho, Sx <-- case sensitive
# plan ใส่ color coded day

import time
import sys
import random

class unavailable:
    def __init__(self, UA_input, weekday, df=df, weekday_date=weekday_date):
        # weekday could be str (eg.'monday') or be integer (eg. 28)   
        if type(weekday) == str: 
            self.dates = [i.day for i in weekday_date[weekday]]
        elif (type(weekday) == list)& (type(weekday[0])==str):
            self.dates = []
            for i in weekday:
                a = [c.day for c in weekday_date[i]]
                self.dates += a
        else:
            self.dates = weekday
            
            
        # UA_input could be  (subspecialty ('Fix')) or list of names
        if UA_input in df.loc[:,'Fix'].unique():
            self.names = [i for i in df[df.loc[:,'Fix'] == UA_input].Name]
        else: 
            self.names = UA_input
#         elif UA_input in df.loc[:,'Name'].unique():
#             self.names = UA_input
            
        self.df = df
        self.weekday_date = weekday_date
        print ('The unavailable dates for {} are day {}'.format(self.names, self.dates))
                
    def UA_bool(self, random_date, random_winner):
        # Will return False if the person is unavailable at a given date  
        if (random_winner in self.names) and (random_date in self.dates):
            return False
        else:
            return True

        
print('checking unavailable Interns...') 
print("Not_preset :{}".format(not_present), "Other OPD: {}".format(other_OPD))
print("There are {0} interns present,".format(len(df)),'Number of days to be filled = {}'.format(len(OFM_dates)))
print('checking unavailable dates...')  

# ภาณุวัฒน์ 15-31 พย elective 1-13 ธค elective

ENT = unavailable('ENT','thursday')
# Panuwat = unavailable(['ภาณุวัฒน์'],'wednesday')
# Panuwat2 = unavailable(['ภาณุวัฒน์'],np.arange(15,31+1))
# Palm = unavailable(UA_input=["ปารินทร์"],weekday=["monday",'wednesday','friday'])
Ckits = unavailable(['ศิรวิชญ์'], ['monday',"thursday"] )
priyaporn = unavailable(['ปรียาภรณ์'], ['wednesday',"friday"] )
Noppadol = unavailable(['นภดล'], ['tuesday',"thursday"] )
Pat = unavailable(['ณัฐภัสสร','นัทธมน'], weekday=[*np.arange(23,29+1)])

def evaluate_conditions(random_date,random_winner):
    conditions = []
    conditions.append(ENT.UA_bool(random_date,random_winner))
#     conditions.append(Panuwat.UA_bool(random_date,random_winner))
#     conditions.append(Panuwat2.UA_bool(random_date,random_winner))
#     conditions.append(Palm.UA_bool(random_date,random_winner))
    conditions.append(Ckits.UA_bool(random_date,random_winner))
    conditions.append(priyaporn.UA_bool(random_date,random_winner))
    conditions.append(Noppadol.UA_bool(random_date,random_winner))
    conditions.append(Pat.UA_bool(random_date,random_winner))
    yield conditions                                                                     

# start randomization

print('\nInitiating randomization..')
"""change month here"""
print('\nเวร OFM เดือนพฤศจิกายน')

count_dict= {}
day_name_dict = {}
start_time = time.time()
seconds = 0
repeat = None
while len(day_name_dict) < len(OFM_dates):
    for date in OFM_dates:
        if (date.day not in day_name_dict.keys()):
            random_winner = random.choice(df.loc[:,'Name'].values)
            current_time = time.time()
            elapsed_time = current_time - start_time
            if (random_winner not in day_name_dict.values())and all(*evaluate_conditions(date.day,random_winner)):
                day_name_dict[date.day] = random_winner
                count_dict[random_winner] = 1
        
            elif elapsed_time > 0.5 and all(*evaluate_conditions(date.day,random_winner)):
                """FIX คนที่อยู่ 2 วัน, just comment it out, when not in use"""
#                 random_winner = random.choice( ["นภสร","ณัชชา"])
                """"""""""""""""""
                day_name_dict[date.day] = random_winner
                count_dict[random_winner] += 1
                name_day = {v: [k for k in day_name_dict if day_name_dict[k] == v] for v in set(day_name_dict.values())}
                repeat = [i for i in name_day if len(name_day[i]) >= 2]
                
    #restart if these names are not in the dictionary            
    if not(all(x in day_name_dict.values() for x in ["นภดล",'ศิรวิชญ์','ปรียาภรณ์'])):
        day_name_dict = {}

if repeat != None:
    print("\n",'คนที่โดนอยู่มากกว่า 1 วันได้แก่', "\n")
    for i in repeat:
        print("{}..อยู่วันที่..{}".format(i,sorted(name_day[i])))
print("\n")
# sort dictionary using ordered dict, and sort by key (t[0])
import collections
day_name_dict = collections.OrderedDict(sorted(day_name_dict.items(), key=lambda t: t[0]))

for i in day_name_dict:
    year = df[df.loc[:,'Name'] == day_name_dict[i]].Year.values
    name = day_name_dict[i]
    print('Day..{}..:.{} {}'.format(i,year, name))



checking unavailable Interns...
Not_preset :['ภาณุวัฒน์', 'สรศักย์', 'ปารินทร์'] Other OPD: []
There are 17 interns present, Number of days to be filled = 15
checking unavailable dates...
The unavailable dates for ENT are day [5, 12, 19, 26]
The unavailable dates for ['ศิรวิชญ์'] are day [2, 9, 16, 23, 30, 5, 12, 19, 26]
The unavailable dates for ['ปรียาภรณ์'] are day [4, 11, 18, 25, 6, 13, 20, 27]
The unavailable dates for ['นภดล'] are day [3, 10, 17, 24, 5, 12, 19, 26]
The unavailable dates for ['ณัฐภัสสร', 'นัทธมน'] are day [23, 24, 25, 26, 27, 28, 29]

Initiating randomization..

เวร OFM เดือนพฤศจิกายน


Day..2..:.['Int3'] นัทธมน
Day..4..:.['Int3'] ณัชชา
Day..5..:.['Int2'] นภสร
Day..6..:.['Int2'] ชญานิศ
Day..9..:.['Int2'] ณิชารีย์
Day..11..:.['Int2'] อภิชญา
Day..12..:.['Int2'] ณัฐชัย
Day..13..:.['Int2'] ปภาวดี
Day..16..:.['Int3'] พุฒิพงศ์
Day..18..:.['Int3'] เอนกโชค
Day..23..:.['Int2'] ปรียาภรณ์
Day..25..:.['Int3'] ศิรวิชญ์
Day..26..:.['Int2'] พิชชาภา
Day..27..:.['Int2'] นภดล
Day..

C:\Users\Mark\anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [30]:
print('\n updating updated_dict \n')
import copy
updated_dict = copy.deepcopy(old_dict)
if old_dict != {}:
    for i in count_dict:
        if i in updated_dict:
            updated_dict[i]+= count_dict[i]
        else:
            updated_dict[i] = 0 
            updated_dict[i]+= count_dict[i]
elif old_dict == {}:
    updated_dict = count_dict

#  print the counting in updated dict
for key in updated_dict:
    year = L_df[L_df.loc[:,'Name'] == key].Year.values
    print('The accumulated number of shifts so far..{} {}..x..{}..times'.format(year,key, updated_dict[key]))


 updating updated_dict 

The accumulated number of shifts so far..['Int2'] นภสร..x..6..times
The accumulated number of shifts so far..['Int2'] พิชชาภา..x..6..times
The accumulated number of shifts so far..['Int2'] ชญานิศ..x..6..times
The accumulated number of shifts so far..['Int2'] สรศักย์..x..6..times
The accumulated number of shifts so far..['Int2'] อภิชญา..x..6..times
The accumulated number of shifts so far..['Int2'] ณัฐชัย..x..6..times
The accumulated number of shifts so far..['Int2'] ณิชารีย์..x..6..times
The accumulated number of shifts so far..['Int3'] ภาสกร..x..6..times
The accumulated number of shifts so far..['Int3'] บุญฤทธิ์..x..4..times
The accumulated number of shifts so far..['Int3'] ภาณุวัฒน์..x..5..times
The accumulated number of shifts so far..['Int3'] ณัชชา..x..6..times
The accumulated number of shifts so far..['Int3'] นัทธมน..x..5..times
The accumulated number of shifts so far..['Int3'] ณัฐภัสสร..x..6..times
The accumulated number of shifts so far..['Int3'] เอนกโชค

### Define functions for saving, backup and reading the files

In [31]:
from os import path
import shutil

# a+ mode will create file if not already existed.
def save_file(file_name, dict):
    with open(file_name, 'wb') as f:
        pickle.dump(dict, f)
        return 'succesfully rewrite file {}'.format(file_name)

def backup(file_name):
    if path.exists(file_name):
        if path.exists(file_name+'_backup'):
            print('backup already existed, will not overwrite file')
        else:
            print('file exists, creating backup:', file_name+'_backup')
            shutil.copy(file_name, file_name+'_backup')
    else:
        print('file did not exist, will not create backup')
    
def read_count(file_name): 
    with open(file_name, 'rb') as f:
        file = pickle.load(f)    
        f.close()
        for key in file:
            print('The number of shifts so far...{} {}..x..{}..times'.format(df[df.loc[:,'Name'] == key].Year.values,key, file[key]))
        
def read_date(file_name):
    with open(file_name, 'rb') as f:
        file = pickle.load(f)    
        f.close()
        for key in file:
            year = L_df[L_df.loc[:,'Name'] == file[key]].Year.values
            print('Day..{}..:..{} {}'.format( key,year,file[key]))


In [32]:
# # เดือนที่แล้วอยู่แทนเอนกโชค
# updated_dict["พุฒิพงศ์"] += 1
# updated_dict["เอนกโชค"] -= 1
# updated_dict

In [33]:
# save to 2 files(_date and _count) and create backup if intending to overwrite file
# day_name_dict is currently orderred dict -> Change back to normal dict- else not readable
def save_all(month):
    date_file = str(month).lower() + '_date'
    count_file = str(month).lower() + '_count'
    try: 
        print('try saving _date file: {}'.format(date_file))
        backup(date_file)
        save_file(date_file, day_name_dict)
        print('success')
    except: 
        print('failed')
        
    try: 
        print('try saving _count file {}'.format(count_file))
        backup(count_file)
        save_file(count_file, updated_dict)
        print('success')
    except: 
        print('failed')

#Read both the count and date files -- just enter the month
def read_files(month):
    date_file = str(month).lower() + '_date'
    count_file = str(month).lower() + '_count'
    print('\n reading {} \n'.format(date_file))
    read_date(date_file)
    print('\n reading {} \n'.format(count_file))
    read_count(count_file)

save_all('November')
read_files('November')

try saving _date file: november_date
file did not exist, will not create backup
success
try saving _count file november_count
file did not exist, will not create backup
success

 reading november_date 

Day..2..:..['Int3'] นัทธมน
Day..4..:..['Int3'] ณัชชา
Day..5..:..['Int2'] นภสร
Day..6..:..['Int2'] ชญานิศ
Day..9..:..['Int2'] ณิชารีย์
Day..11..:..['Int2'] อภิชญา
Day..12..:..['Int2'] ณัฐชัย
Day..13..:..['Int2'] ปภาวดี
Day..16..:..['Int3'] พุฒิพงศ์
Day..18..:..['Int3'] เอนกโชค
Day..23..:..['Int2'] ปรียาภรณ์
Day..25..:..['Int3'] ศิรวิชญ์
Day..26..:..['Int2'] พิชชาภา
Day..27..:..['Int2'] นภดล
Day..30..:..['Int3'] ณัฐภัสสร

 reading november_count 

The number of shifts so far...['Int2'] นภสร..x..6..times
The number of shifts so far...['Int2'] พิชชาภา..x..6..times
The number of shifts so far...['Int2'] ชญานิศ..x..6..times
The number of shifts so far...['Int2'] สรศักย์..x..6..times
The number of shifts so far...['Int2'] อภิชญา..x..6..times
The number of shifts so far...['Int2'] ณัฐชัย..x..6.